In [3]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [4]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

In [5]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

#pragma warning disable SKEXP0050
var bingConnector = new BingConnector(bingSearchAPIKey);
var bingSearchPlugin = new WebSearchEnginePlugin(bingConnector);

semanticKernel.ImportPluginFromObject(bingSearchPlugin, "bing");
#pragma warning restore SKEXP0050

In [23]:
var promptTemplateString = "What are the current {{$decisionTopic}} being used by companies?";
var bingSearchPluginImport = semanticKernel.Plugins["bing"]["search"];

var kernelArguments = new KernelArguments()
{
    ["decisionTopic"] = "simple decision frameworks"
};
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(new PromptTemplateConfig(promptTemplateString));
var renderedPrompt = await promptTemplate.RenderAsync(semanticKernel, kernelArguments);

var kernelArgumentsBing = new KernelArguments()
{
    ["query"] = renderedPrompt
};
var bingResult = await semanticKernel.InvokeAsync(bingSearchPluginImport, kernelArgumentsBing);

Console.Write(bingResult);

["3 frameworks for collective decision making. Group decision-making is a must when dealing with sensitive topics or subjects that a lot of people have an interest in. A substantial part of what stakeholder management means is the need to share accountability. Frameworks help a great deal in organizing people and giving clarity on objectives.","4 Cynefin framework. 5 Xanax for decision-making. 6 Dimensionality reduction. 7 WRAP framework. 8 Wrapping up. The Logic or Decision Tree is a critical thinking framework based on root-cause analysis. The essence of the framework lies in a visual representation of a problem decomposed into small chunks.","Frameworks are the lifeline of every case – whether in real life or in a case interview.They help navigate complex business scenarios in a simple, structured, and pragmatic manner while being rooted in hypotheses and concepts like 80:20 and the pyramid principle.One of the important types of cases you will experience in case interviews is “deci